# Cross referencing Google location history with MOH COVID-19 exposure locations
## Important notice!
> This code was **not** created with accuracy in mind, this is a POC and I really wanted to try my hand with pandas and used COVID-19 as an excuse. Please **do not** rely on the results.
I know for a fact that some locations that are marked as "clear" (location history indicated not in proximity of location) also have additional date and time information in the comment field.

## Notes
- This code calculates location history distance from exposure locations without any distance considuration.
- The code is not just **not optimized**, it bad, slow and unreadable.
- First time using pandas, didn't read the documentation. Also, not a programmer.
- It does work...

## Known bugs
- MOH timestamp fields are untrustworthy, therefore a combination of the timestamp and a stayTimes string  is used.
- If a stayTimes string is ends before it begins (such as "09:00 - 02:00") no corrections are made to the date and location is ignored.
- If the stayTimes string is "לא ידוע" ("Unknown"), there is not attempt done to select the full time range.
- MOH like to add additional exposure times in the comment field, I ignore them.

## Testing
With the supplied myData.json and govData.json, there should be 1169 cases of unknown locations maked orange on the map. Out of the 132 known locations, 130 should be ok (green) and 2 should be too close to an exposure location and marked with a red marker on the map (one in Ra'anana
 and another at the Ruppin academic center).

## Logic behind the code
    None.

In [ ]:
# imports
import numpy as np
import pandas as pd
import requests
import datetime as dt
import geopy.distance
import json
import os
from ipywidgets import IntSlider
from ipywidgets.embed import embed_minimal_html
from ipyleaflet import Map, FullScreenControl, Marker, basemaps, AwesomeIcon
import itertools


%matplotlib notebook

# Make page wider, better view of tabular data
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### Asset acquisition
This will download the latest MOH exposure locations file. Also, make sure the "Location History.json" file from google
is located in the same directory under the name "myData.json". If I did not forget, there should be a sample "myData.json" file in the repository to run this notebook without actual data.

In [ ]:
# COVID-19 exposure locations from MOH
GOV_CASE_LOC_URL = "https://gisweb.azureedge.net/Points.json"

def downloadFile(url, path):
    """
    Download a file and save it.
    """
    print(f"Requesting {url}")
    res = requests.get(url, stream=True)
    length = res.headers.get("content-length")
    print(f"Downloading")
    with open(path, 'wb') as f:
        if length is None:
            print("No content length.")
            f.write(res.content)
        else:
            length = int(length)
            dl_length = 0
            print(f"Total size: {length}")
            for data in res.iter_content(chunk_size=4096):
                dl_length += len(data)
                f.write(data)
            print(f"Downloaded: {dl_length}")

# Uncomment to download newest MOH dataset
#downloadFile(GOV_CASE_LOC_URL, 'govData.json')
if not os.path.isfile('myData.json'):
    # Copy the "Location History.json" file from Google takeout to here, rename it "myData.json"
    raise Exception("Missing location history file (myData.json). ")

### Google data normalization 
Load the google location history and put it into a known DataFrame with a known format.

In [ ]:
%%time
# Transform Google location history data to a known dataframe
# Note to self: Google Location History files are big, dont load with "pd.load_json"
loch_data = json.load(open("myData.json", "r"))
loch_df = pd.DataFrame.from_dict(loch_data)
print(f"Number of Google location history points loaded: {loch_df.shape[0]:,}.")

# Extract timestamp, long, lat, accuracy and datetime from Location History.
# For future use and debugging, the timestamp is also converted to a datetime.
# Note, timestamp is in mS, long and lat must be devided by 1e7.
loch_df['timestamp'] = loch_df.locations.map(lambda x: int(x['timestampMs'])) # Not really used
loch_df['longitude'] = loch_df.locations.map(lambda x: x['longitudeE7']/1e7)
loch_df['latitude'] = loch_df.locations.map(lambda x: x['latitudeE7']/1e7)
loch_df['accuracy'] = loch_df.locations.map(lambda x: x['accuracy'])
loch_df['datetime'] = loch_df.timestamp.map(lambda x: dt.datetime.fromtimestamp(x/1000))

# Drop the rest of the dataset that is now irrelevent.
loch_df= loch_df.drop(labels=['locations'], axis=1, inplace=False)

# Display subset of data
print(f"Timeline: from {loch_df.datetime.min().strftime('%d/%m/%Y %H:%M')} to {loch_df.datetime.max().strftime('%d/%m/%Y %H:%M')}")
print(f"Accuracy: min {loch_df.accuracy.min()}, max {loch_df.accuracy.max()}")
display(loch_df)

### MOH data normalization 
A very band and lazy attempt at normalizing the MOH dataset, there are so many edge cases.
In order to get a (maybe?) correct duration time, the date component of the timestamp is used with a strptime parsed string of the 
time component. The only edge case which is handled here is when the end datetime time component is 00:00, I manualy add a delta of 1 day to the datetime (as the MOH dont advance the date).

In [ ]:
%%time
def _convertTime(time, index, start_timestamp_ms, end_timestamp_ms):
    """
    A horrible function to try and convert the disguising MOH time data to something useful.
    """
    try:
        parsed_time = dt.datetime.strptime(time.split('-')[index].strip(), "%H:%M").time()
        parsed_start_date = dt.datetime.fromtimestamp(start_timestamp_ms/1000).date()
        parsed_end_date = dt.datetime.fromtimestamp(end_timestamp_ms/1000).date()
        if index == 0:
            parsed_date = parsed_start_date
        else:
            parsed_date = parsed_end_date
        
        # Dumb MOH programmers use 00:00 to denote midnight without changing the date
        # So, if we are looking for the end date, and the time is 00:00 and the dates did not change, fix it.
        # This still does not fix the "09:00 - 02:00" problem.
        if (index == 1) and (parsed_time == dt.time(0,0)) and (parsed_start_date == parsed_end_date):
            delta = dt.timedelta(days=1)
        else:
            delta = dt.timedelta(days=0)
        
        return dt.datetime.combine(parsed_date + delta, parsed_time)
    except Exception as ex:
        # A date of 01/01/3000 marks a parse error.
        return dt.datetime(3000,1,1,0,0,0)

# Transform MOH data to a known dataframe
moh_data = json.load(open("govData.json",  "r")) 
moh_df = pd.DataFrame.from_dict(moh_data)  
print(f"Number of MOH points loaded: {moh_df.shape[0]:,}.")

# Extract id, long, lat, type, name, place, comments, fixed format times
# Not exactly sure what "type" is, assuming will allow polygons/shapes in the future.
moh_df['id'] = moh_df['features'].map(lambda x: x['id'])
moh_df['longitude'] = moh_df['features'].map(lambda x: x['geometry']['coordinates'][0])
moh_df['latitude'] = moh_df['features'].map(lambda x: x['geometry']['coordinates'][1])
moh_df['type'] = moh_df['features'].map(lambda x: x['geometry']['type'])
moh_df['iname'] = moh_df['features'].map(lambda x: x['properties']['Name'])
moh_df['place'] = moh_df['features'].map(lambda x: x['properties']['Place'])
moh_df['comments'] = moh_df['features'].map(lambda x: x['properties']['Comments'])
# Note to self: The time convertion can be done better, much much better.
moh_df['start_datetime'] = moh_df['features'].map(lambda x: _convertTime(x['properties']['stayTimes'], 0, x['properties']['fromTime'], x['properties']['toTime']))
moh_df['end_datetime'] = moh_df['features'].map(lambda x: _convertTime(x['properties']['stayTimes'], 1, x['properties']['fromTime'], x['properties']['toTime']))
# Debug datetime columns, not used for any referencing.
moh_df['debug_stayTimes'] = moh_df['features'].map(lambda x: x['properties']['stayTimes'])
moh_df['debug_fromTime'] = moh_df['features'].map(lambda x: dt.datetime.fromtimestamp(x['properties']['fromTime']/1000))
moh_df['debug_toTime'] = moh_df['features'].map(lambda x: dt.datetime.fromtimestamp(x['properties']['fromTime']/1000))

# Drop the rest of the dataset
moh_df = moh_df.drop(labels=['features'], axis=1, inplace=False)

# Dispay subset of data and count and subset of bad time information
print(f"Timeline: from {moh_df.start_datetime.min().strftime('%d/%m/%Y %H:%M')} to {moh_df.end_datetime[moh_df.start_datetime < dt.datetime(3000,1,1,0,0,0)].max().strftime('%d/%m/%Y %H:%M')}")  
bad_datetimes = moh_df[moh_df.start_datetime >= moh_df.end_datetime]
no_bad_datetimes = bad_datetimes.shape[0]
unknown_datetimes = moh_df[moh_df.start_datetime == dt.datetime(3000,1,1,0,0,0)]
no_unknown_datetimes = unknown_datetimes.shape[0]
print(f"Bad start/end datetimes: {no_bad_datetimes}") 
print(f"Unknown datetimes: {no_unknown_datetimes}") 
print(f"Total valid datapoints: {moh_df.shape[0] - no_bad_datetimes - no_unknown_datetimes}")
print("-"*40)
print("Subset of valid datapoints:")
display(moh_df)
print("Subset of bad start/end datapoints:")
display(bad_datetimes)
print("Subset of unknown datetime datapoints:")
display(unknown_datetimes)


In [ ]:
# Make a slice of only the relevant  dates out of the location history DataFrame.
loch_slice = loch_df[(loch_df.datetime >=  moh_df.start_datetime.min()) & (loch_df.datetime <= moh_df.end_datetime[moh_df.end_datetime < dt.datetime(3000,1,1,0,0,0)].max())]
print(f"Cross referencing with location history from {loch_slice.datetime.min().strftime('%d/%m/%Y %H:%M')} to {loch_slice.datetime.max().strftime('%d/%m/%Y %H:%M')}")

## The super duper deoptimized cross referencing loops
I cant really explain this, but this is what I think I did:
1. Create master results DataFrame.
2. For each row of the MOH exposure points:
    - Do some time sanity checks, if time is wierd, put it aside.
    - Get the time relevant slice of location history.
    - For each location history point, mesure the distance and add to a private dataframe
    - Take the results with minimum distance and the minimum accuracy and combine them into a sngle result row in the master results DataFrame.
3. Display subset of master results DataFrame.

> Note: There is no reason to create the internal DataFrame, I did it this way so later I could use the DataFrames as sample data to play with pandas.

In [ ]:
%%time
TIME_BUFFER = dt.timedelta(minutes=0)
# Counters for bad datapoints
# Note: These were counters, now they are lists.
counters = { 'unknown_datetime' : [], 'bad_datetime' : [], 'missing_results' : [], 'debug_timestamp' : [], 'totally_failed' : []}
# Total dataset size
frame_size = moh_df.shape[0]

# Do I really need to create this DataFrame? No.
results_df = pd.DataFrame(columns=['incident_id', 'incident_time', 'incident_location', 'incident_name','incident_place', 'incident_comments', 'min_distance_location','min_distance_accuracy', 'min_distance_distance', 'min_accuracy_accuracy'])
for inc_idx, inc_row in moh_df.iterrows():
    # Everything is in a try, whay? because MOD data can lie!
    try:
        # Time buffers! may be useful in the future
        search_start_datetime = inc_row.start_datetime - TIME_BUFFER
        search_end_datetime = inc_row.end_datetime + TIME_BUFFER
        
        # Some sanity checks
        if inc_row.start_datetime > inc_row.end_datetime:
            counters['bad_datetime'].append(inc_row)
            continue
        if inc_row.start_datetime == dt.datetime(3000,1,1,0,0,0):
            counters['unknown_datetime'].append(inc_row)
            continue
            # Select the whole timerange
            search_start_datetime = datetime.combine(search_start_datetime.date(), dt.time(0,0))
            search_end_datetime = datetime.combine(search_end_datetime.date() + dt.timedelta(days=1), dt.time(0,0))
        # Debug, just to see how many timestams are *maybe* correct
        if inc_row.debug_fromTime != inc_row.debug_toTime:
            counters['debug_timestamp'].append(inc_row)

        # Get all locations from location history that corrspond to the datetime
        loch_results = loch_slice[(loch_slice.datetime >= search_start_datetime ) & (loch_slice.datetime <= search_end_datetime)]
        if loch_results.empty:
            # No results, manual verification :(
            counters['missing_results'].append(inc_row)
            continue

        # We can only set a distance and index, but it's easy to run slicings on a clean DataFrame
        measured_distances = pd.DataFrame(columns=['index','datetime', 'distance', 'latitude', 'longitude', 'accuracy'])
        for loc_idx, loc_row in loch_results.iterrows():
            distance = geopy.distance.geodesic((inc_row.longitude, inc_row.latitude),(loc_row.longitude, loc_row.latitude))
            measured_distances = measured_distances.append({ 'index' : loc_idx,
                                                           'datetime' : loc_row.datetime,
                                                           'distance' : distance.km,
                                                           'latitude' : loc_row.longitude,
                                                           'longitude' : loc_row.latitude,
                                                           'accuracy' : loc_row.accuracy}, ignore_index=True)
        
        min_distance_df = measured_distances.sort_values(by=['distance'], ).head(1)
        min_accuracy_df = measured_distances.sort_values(by=['accuracy'], ).head(1)
        
        results_df = results_df.append({'incident_id' : inc_row.id, 
                                  'incident_time' : str(f'{inc_row.start_datetime.strftime("%d/%m/%Y %H:%M")} - {inc_row.end_datetime.strftime("%d/%m/%Y %H:%M")}'), 
                                  'incident_location' : (inc_row.latitude, inc_row.longitude), 
                                  'incident_name' : inc_row.iname,
                                  'incident_place' : inc_row.place, 
                                  'incident_comments' : inc_row.comments,
                                   # I dont know why (yet) but I needed to use here .item()
                                  'min_distance_location' : (min_distance_df.latitude.item(), min_distance_df.longitude.item()), 
                                  'min_distance_accuracy' : min_distance_df.accuracy.item(),
                                  'min_distance_distance' : min_distance_df.distance.item(),
                                  'min_accuracy_location' : (min_accuracy_df.latitude.item(), min_accuracy_df.longitude.item()), 
                                  'min_accuracy_distance' : min_accuracy_df.distance.item(),
                                  'min_accuracy_accuracy' : min_accuracy_df.accuracy.item()}, ignore_index=True)
    except Exception as ex:
        # Yep, the data lied.
        print(f"You have 1 new exception: {ex}")
        counters['totally_failed'].append(inc_row)
    
    # A poor mans progress meter.
    if inc_idx % 100 == 0:
        print(f"\rProcessed {inc_idx}/{frame_size} exposure incidents (estimate)", end='')
print(" Done")

print(f"Total incidents checked: {results_df.shape[0]}")
print(f"Incidents with unknown time skipped: {len(counters['unknown_datetime'])}")
print(f"Incidents with bad time skipped: {len(counters['bad_datetime'])}")
print(f"Incidents with missing results: {len(counters['missing_results'])}")
print(f"Incidents witch just failed for some reason: {len(counters['totally_failed'])}")
print(f"Debug timestamp incidents: {len(counters['debug_timestamp'])}")
# Show some results    
display(results_df.sort_values(by=['min_distance_distance']))



## Display a map
Display a map swith all MOH locations on it.
    - Red icons - Maybe exposed.
    - Orange icons - No cross-referencing done.
    - Green - Probably not exposed (again, hidden additional time data in the comments field).

Due to not grouping of the markers (as I want to see them all with their colors) the map is **very** sluggish. I did not intent on adding a map, but it's Jupyter and I can. 


In [ ]:
%%time
CRITICAL_DISTANCE_KM = 1.0
# Create the map
m = Map(zoom=7, center=[31.771959, 35.217018],scroll_wheel_zoom=True,basemap = basemaps.OpenStreetMap.Mapnik)
# Create the icons
red_icon = AwesomeIcon(marker_color='red', name='exclamation-triangle')
green_icon = AwesomeIcon(marker_color='green', name='check')
orange_icon = AwesomeIcon(marker_color='orange', name='flag')

# Add known locations markers
known_count = len([m.add_layer(Marker(z_index_offset=50 if row.min_distance_distance < CRITICAL_DISTANCE_KM else 0,icon=red_icon if row.min_distance_distance < CRITICAL_DISTANCE_KM else green_icon,draggable=False, location=row.incident_location, title=f'מיקום: {row.incident_place}\nהערות: {row.incident_comments}\nזמן: {row.incident_time}\nמרחק: {row.min_distance_distance:0.4f} ק"מ\nדיוק: {row.min_distance_accuracy}')) for idx, row in results_df.iterrows()])
# Add unknown/bad/missing markers
unknown_count = len([m.add_layer(Marker(z_index_offset=25, icon=orange_icon, draggable=False, location=(row.latitude, row.longitude), title=f'מיקום: {row.place}\nהערות: {row.comments}\nזמן: {row.start_datetime.strftime("%d/%m/%Y %H:%M")} - {row.end_datetime.strftime("%d/%m/%Y %H:%M")}')) for row in itertools.chain.from_iterable(counters.values())])
# Alow full screen
m.add_control(FullScreenControl())
# Display some info 
print(f"Added {known_count} known markers.")
print(f"Added {unknown_count} unknown markers.")
print(f"Total markers on map: {known_count + unknown_count}/{frame_size}")
display(m)


## Save the data and the map
Saves the date and map, incase we want it later. 

In [ ]:
results_df.sort_values(by=['min_distance_distance']).to_html(dt.datetime.now().strftime("data/data%Y%m%d_%H%M.html"))
embed_minimal_html(dt.datetime.now().strftime("maps/map%Y%m%d_%H%M.html"), views=[m], title='Exposure Map')

The end.